In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from scipy.sparse import csr_matrix
from sklearn.metrics import accuracy_score
import torch
import os
import seaborn as sns
from torch.utils.data import DataLoader,TensorDataset
import pickle
import json
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score
from sae.preprocess import get_meta_info,get_freq_emmbeddings, get_tf_idfs_emmbeddings,get_bert_embeddings
from sae.models import DEEPmodel_2
import time
import joblib
%autoreload 2

c:\Users\natha\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data_all = pd.read_csv('ratings_formatted.csv')
data_train, data_test = train_test_split(data_all, test_size=0.2, random_state=42)


In [4]:
Freqmodel = joblib.load('freq_model.pkl')
idfmodel = joblib.load('tfidf_model.pkl')

In [5]:
mode = "summary"
X_emb_freq,model_freq = get_freq_emmbeddings(data_train[mode].values,model=Freqmodel)
X_emb_idf,model_idf = get_tf_idfs_emmbeddings(data_train[mode].values,model=idfmodel)
X_test_emb_freq,_ = get_freq_emmbeddings(data_test[mode].values,model=Freqmodel)
X_test_emb_idf,_ = get_tf_idfs_emmbeddings(data_test[mode].values,model=idfmodel)

In [6]:
y_train_rating= np.load('y_train_rating.npy')
y_test_rating = np.load('y_test_rating.npy')

In [7]:
y_train_rating

array([5, 5, 4, ..., 5, 5, 5])

In [8]:
X_train_meta=data_train[['auteur_note','auteur_nb','genre_note','genre_nb','editor_note','editor_nb']].values
X_test_meta=data_test[['auteur_note','auteur_nb','genre_note','genre_nb','editor_note','editor_nb']].values
X_train_freq = np.concatenate((X_train_meta,X_emb_freq),axis=1)
X_test_freq = np.concatenate((X_test_meta,X_test_emb_freq),axis=1)
X_train_idf = np.concatenate((X_train_meta,X_emb_idf),axis=1)
X_test_idf = np.concatenate((X_test_meta,X_test_emb_idf),axis=1)
#X_train_bert = np.concatenate((X_train_meta,X_train_bert),axis=1)
#X_test_bert = np.concatenate((X_test_meta,X_test_bert),axis=1)
print("freq :",X_train_freq.shape)
print("idf :",X_train_idf.shape)
#print("bert :",X_train_bert.shape)

freq : (2399668, 1006)
idf : (2399668, 1006)


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## frec

In [10]:
print(np.shape(y_train_rating))
print(np.shape(X_train_freq))

(2399668,)
(2399668, 1006)


In [11]:
batch_size = 4096*4
train_dataset_bert_sent = TensorDataset(torch.tensor(X_train_freq),torch.tensor(y_train_rating, dtype=torch.long))
test_dataset_bert_sent = TensorDataset(torch.tensor(X_test_freq),torch.tensor(y_test_rating, dtype=torch.long))
train_loader_sent = DataLoader(train_dataset_bert_sent, batch_size=batch_size, shuffle=True)
test_loader_sent = DataLoader(test_dataset_bert_sent, batch_size=batch_size, shuffle=False)

In [12]:
print(train_loader_sent)

In [43]:
import torch

print("CUDA disponible :", torch.cuda.is_available())
print("Nombre de GPU :", torch.cuda.device_count())
print("Nom du GPU :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Aucun GPU détecté")
print("Index du GPU utilisé :", torch.cuda.current_device() if torch.cuda.is_available() else "N/A")


CUDA disponible : True
Nombre de GPU : 1
Nom du GPU : NVIDIA GeForce RTX 3090
Index du GPU utilisé : 0


In [13]:
nb_epoch = 10
#model = DEEPmodel(input_size=774, feature_size=256, nb_class=5, nb_couche=1).to(device)
model = DEEPmodel_2(input_size=1006, features = [512,512,512,512,256], sub_features =[256,256], nb_class=5)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(nb_epoch):
    model.train()
    y_pred_all = []
    nb = 0
    total = 0
    
    for i, (x, y) in enumerate(train_loader_sent):
        print(".",end="")
        x, y = x.to(device).float(), y.to(device)  # Move data to GPU
        optimizer.zero_grad()
        y_pred = model(x)
        y_pred_all.append(y_pred.detach().cpu())  # Move to CPU for later operations
        loss = loss_fn(y_pred, y-1)
        nb += 1
        total += y.size(0)
        loss.backward()
        optimizer.step()
    print()
    print(f"epoch {epoch} :  loss : {loss.item():0.3f}")
    
    y_pred_all = torch.cat(y_pred_all)
    acc_train = accuracy_score(y_train_rating-1 , y_pred_all.argmax(1).numpy())
    
    model.eval()
    with torch.no_grad():
        y_pred = []
        for i, (x, y) in enumerate(test_loader_sent):
            x = x.to(device).float()  # Move test data to GPU
            y_pred_batch = model(x).argmax(1)
            y_pred.append(y_pred_batch.cpu())  # Move to CPU for evaluation
        
        y_pred = torch.cat(y_pred)
        acc_test = accuracy_score(y_test_rating-1, y_pred.numpy())
    
    print(f"epoch {epoch} :   test : {acc_test:0.3f}")

...................................................................................................................................................
epoch 0 :  loss : 0.975
epoch 0 :   test : 0.638
...................................................................................................................................................
epoch 1 :  loss : 0.946
epoch 1 :   test : 0.642
...................................................................................................................................................
epoch 2 :  loss : 0.965
epoch 2 :   test : 0.644
...................................................................................................................................................
epoch 3 :  loss : 0.942
epoch 3 :   test : 0.645
...................................................................................................................................................
epoch 4 :  loss : 0.920
epoch 4 :   test : 0.646
...............

In [14]:
torch.save(model.state_dict(), "Re_Deep_freq.pth")

## Tf idf

In [15]:
print(np.shape(y_train_rating))
print(np.shape(X_train_idf))

(2399668,)
(2399668, 1006)


In [16]:
batch_size = 4096*4
train_dataset_bert_sent = TensorDataset(torch.tensor(X_train_idf),torch.tensor(y_train_rating, dtype=torch.long))
test_dataset_bert_sent = TensorDataset(torch.tensor(X_test_idf),torch.tensor(y_test_rating, dtype=torch.long))
train_loader_sent = DataLoader(train_dataset_bert_sent, batch_size=batch_size, shuffle=True)
test_loader_sent = DataLoader(test_dataset_bert_sent, batch_size=batch_size, shuffle=False)

In [17]:
nb_epoch = 10
#model = DEEPmodel(input_size=774, feature_size=256, nb_class=5, nb_couche=1).to(device)
model = DEEPmodel_2(input_size=1006, features = [512,512,512,512,256], sub_features =[256,256], nb_class=5)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(nb_epoch):
    model.train()
    y_pred_all = []
    nb = 0
    total = 0
    
    for i, (x, y) in enumerate(train_loader_sent):
        print(".",end="")
        x, y = x.to(device).float(), y.to(device)  # Move data to GPU
        optimizer.zero_grad()
        y_pred = model(x)
        y_pred_all.append(y_pred.detach().cpu())  # Move to CPU for later operations
        loss = loss_fn(y_pred, y-1)
        nb += 1
        total += y.size(0)
        loss.backward()
        optimizer.step()
    print()
    print(f"epoch {epoch} :  loss : {loss.item():0.3f}")
    
    y_pred_all = torch.cat(y_pred_all)
    acc_train = accuracy_score(y_train_rating-1 , y_pred_all.argmax(1).numpy())
    
    model.eval()
    with torch.no_grad():
        y_pred = []
        for i, (x, y) in enumerate(test_loader_sent):
            x = x.to(device).float()  # Move test data to GPU
            y_pred_batch = model(x).argmax(1)
            y_pred.append(y_pred_batch.cpu())  # Move to CPU for evaluation
        
        y_pred = torch.cat(y_pred)
        acc_test = accuracy_score(y_test_rating-1, y_pred.numpy())
    
    print(f"epoch {epoch} :   test : {acc_test:0.3f}")

...................................................................................................................................................
epoch 0 :  loss : 0.958
epoch 0 :   test : 0.637
...................................................................................................................................................
epoch 1 :  loss : 0.966
epoch 1 :   test : 0.641
...................................................................................................................................................
epoch 2 :  loss : 0.944
epoch 2 :   test : 0.643
...................................................................................................................................................
epoch 3 :  loss : 0.959
epoch 3 :   test : 0.644
...................................................................................................................................................
epoch 4 :  loss : 0.939
epoch 4 :   test : 0.645
...............

In [18]:
torch.save(model.state_dict(), "Re_Deep_idf.pth")